In [63]:
%matplotlib inline

import colorsys
import random
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from collections import Counter
from bokeh.plotting import figure, show
from bokeh.resources import CDN
from bokeh.io import output_notebook
output_notebook( resources=CDN )
import math

pd.set_option('max_colwidth', 200)

Loading BokehJS ...

In [50]:
# Tube
lines       = pd.read_csv('london.lines.csv', index_col=0)
stations    = pd.read_csv('london.stations.csv', index_col=0)
stationsRein= pd.read_csv('london.stations.reindexed.csv', index_col=0)
connections = pd.read_csv('london.connections.csv')

# Bikes
rawData       = pd.read_csv('bikes/london-bikes-coords.csv', index_col=0)

In [51]:
tubeGraph = nx.Graph()

for connection_id, connection in connections.iterrows():
    station1_name = stations.ix[connection['station1']]['name']
    station2_name = stations.ix[connection['station2']]['name']
    tubeGraph.add_edge(station1_name, station2_name, time = connection['time'])
    
#add the connection between Bank and Monument manually
tubeGraph.add_edge('Bank', 'Monument', time = 1)
# nx.draw(tubeGraph)

/usr/local/Cellar/ipython/6.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.
/usr/local/Cellar/ipython/6.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


In [52]:
nx.shortest_path(tubeGraph, 'Oxford Circus', 'Canary Wharf', weight='time')

['Oxford Circus',
 'Tottenham Court Road',
 'Holborn',
 'Chancery Lane',
 "St. Paul's",
 'Bank',
 'Shadwell',
 'Wapping',
 'Rotherhithe',
 'Canada Water',
 'Canary Wharf']

In [53]:
# Print Tube Graph
print(tubeGraph)

In [31]:
normed = stations[['longitude', 'latitude']]
normed = normed - normed.min()
normed = normed / normed.max()
locations = dict(zip(stations['name'], normed[['longitude', 'latitude']].values))

p = figure(
    x_range = (.4,.7),
    y_range = (.2,.5),
    height= 700,
    width= 900,
)
for edge in tubeGraph.edges():
    p.line( 
        x= [locations[pt][0] for pt in edge],
        y= [locations[pt][1] for pt in edge],
    )

for node in tubeGraph.nodes():
    x = [locations[node][0]]
    y = [locations[node][1]]
    p.circle(
        x, y, 
        line_alpha=0)
    p.text(
        x, y, 
        text = {'value':node}, 
        text_font_size = str(10) +"pt",
        text_align='center',
        text_font_style='bold')
    
show(p)

In [66]:
nx.algorithms.connectivity.cuts.minimum_node_cut(tubeGraph)

{'Warwick Avenue'}

In [67]:
nx.algorithms.connectivity.stoerwagner.stoer_wagner(tubeGraph)

(1,
 (['Upminster'],
  ['Chiswick Park',
   'Stamford Brook',
   'Bond Street',
   'West India Quay',
   'Northwick Park',
   'Arnos Grove',
   'Highbury & Islington',
   'Island Gardens',
   'South Ruislip',
   'Canons Park',
   'Wimbledon Park',
   'West Hampstead',
   'Camden Town',
   'Stockwell',
   'Hounslow West',
   'Woodside Park',
   'Moorgate',
   'North Acton',
   'Aldgate',
   'Beckton',
   'Elverson Road',
   'Putney Bridge',
   'New Cross Gate',
   'Cockfosters',
   'Bromley-By-Bow',
   'South Woodford',
   'Bounds Green',
   'Richmond',
   'Upminster Bridge',
   'Colindale',
   'Highgate',
   'West Ham',
   'Morden',
   'Southfields',
   'Tottenham Court Road',
   'Brixton',
   'Warren Street',
   'Beckton Park',
   "King's Cross St. Pancras",
   'Upton Park',
   'South Kensington',
   'Alperton',
   'Wanstead',
   'Becontree',
   'Plaistow',
   'Canary Wharf',
   'Upney',
   'Hounslow East',
   "Shepherd's Bush (C)",
   'Hampstead',
   'Fairlop',
   'Chalk Farm',
   'R

In [61]:
# BIKES
uniqueBikeLocations = {}
uniqueBikeCount = 0

for id, info in rawData.iterrows():
    if info["StartStation Name"] not in uniqueBikeLocations:
        name = info["StartStation Name"]
        lat = info["StartLatitude"]
        long = info["StartLongitude"]
        uniqueBikeLocations[name] = (lat, long)
        uniqueBikeCount += 1
        
    if info["EndStation Name"] not in uniqueBikeLocations:
        name = info["EndStation Name"]
        lat = info["EndLatitude"]
        long = info["EndLongitude"]
        uniqueBikeLocations[name] = (lat, long)
        uniqueBikeCount += 1
        
print("UniqueBikeCount: ",uniqueBikeCount)

UniqueBikeCount:  733


In [62]:
# list of unique bike locations ("stations") and their coordinates
pd.DataFrame.from_dict(uniqueBikeLocations, orient="index", columns=['latitude', 'longitude'])

,latitude,longitude
"Windsor Terrace, Hoxton",51.529239,-0.093476
"Royal London Hospital, Whitechapel",51.518222,-0.058518
"Newton Street, Covent Garden",51.516573,-0.121343
"Aldersgate Street, Barbican",51.520091,-0.097608
"West Smithfield Rotunda, Farringdon",51.518248,-0.100737
"Lisson Grove, St. John's Wood",51.524779,-0.168315
"Finsbury Leisure Centre, St. Luke's",51.526022,-0.095231
"York Hall, Bethnal Green",51.530140,-0.055168
"Devons Road, Bow",51.522163,-0.018298
"Driffield Road, Old Ford",51.533171,-0.036762


In [145]:
# Calcuates mean coordinates based on Tube Stations

latcoors = stationsRein['latitude']
longcoors = stationsRein['longitude']
latmean = latcoors.mean()
longmean = longcoors.mean()

# meancoors = (latmean, longmean)

# Center of London
meancoors = (51.510776, -0.115638)

print("Mean Coordinates: ", meancoors)
longitudeBoundSize = 0.2 # Mess with these for bigger radius
latitudeBoundSize = 0.2  # Mess with these for bigger radius

randLat1 = meancoors[0] + (random.uniform(0,1)*latitudeBoundSize - (latitudeBoundSize/2))
randLong1 = meancoors[1] + (random.uniform(0,1)*longitudeBoundSize - (longitudeBoundSize/2))
randLat2 = meancoors[0] + (random.uniform(0,1)*latitudeBoundSize - (latitudeBoundSize/2))
randLong2 = meancoors[1] + (random.uniform(0,1)*longitudeBoundSize - (longitudeBoundSize/2))

print("Random origin location:        ", randLat1,"," ,randLong1)
print("Random destination location:   ", randLat2,",", randLong2)

Mean Coordinates:  (51.510776, -0.115638)
Random origin location:         51.56196001727023 , -0.14691236779801198
Random destination location:    51.527678828794734 , -0.18236604319331579


In [146]:
# Calculates the closest TUBE stations to the random coordinates

#calculate distances to all stations in array from co-ordinates
tubeDistances1 = []
tubeDistances2 = []
for i in range(1, len(stationsRein['longitude'])):
    distance1 = math.sqrt((randLat1 - stationsRein['latitude'].iloc[i-1])**2 + (randLong1 - stationsRein['longitude'].iloc[i-1])**2)
    distance2 = math.sqrt((randLat2 - stationsRein['latitude'].iloc[i-1])**2 + (randLong2 - stationsRein['longitude'].iloc[i-1])**2)
    #print(stationsRein['name'].iloc[i-1])
    #print(i)
    #print(stationsRein['latitude'].iloc[i-1])
    #print(distance1)
    
    tubeDistances1.append(distance1)
    tubeDistances2.append(distance2)

#pick shorest for each co-ordinate
station1 = tubeDistances1.index(min(tubeDistances1))
station2 = tubeDistances2.index(min(tubeDistances2))
#print(station1)
#print(distances1[station1])
print("Nearest station to Random Origin:", stationsRein['name'][station1+1], stationsRein['latitude'][station1+1], stationsRein['longitude'][station1+1])
print("Nearest station to Random Destination:", stationsRein['name'][station2+1],stationsRein['latitude'][station2+1], stationsRein['longitude'][station2+1])



Nearest station to Random Origin: Tufnell Park 51.5567 -0.1374
Nearest station to Random Destination: Maida Vale 51.53 -0.1854


In [147]:
# Calculate the closest BIKE station to the random coordinates

# Calculate distances to all stations in array from co-ordinates
shortestBikeOriginName = ""
shortestBikeDestinationName = ""
shortestBikeOriginValue = 9999
shortestBikeDestinationValue = 9999

for bikeLoc in uniqueBikeLocations:
    bName = bikeLoc
    bLat  = uniqueBikeLocations[bikeLoc][0]
    bLong = uniqueBikeLocations[bikeLoc][1]
    distanceFromRandomToOrigin      = math.sqrt((randLat1 - bLat)**2 + (randLong1 - bLong)**2)
    distanceFromRandomToDestination = math.sqrt((randLat2 - bLat)**2 + (randLong2 - bLong)**2)
    
    if distanceFromRandomToOrigin < shortestBikeOriginValue:
        shortestBikeOriginName = bName
        shortestBikeOriginValue = distanceFromRandomToOrigin
        
    if distanceFromRandomToDestination < shortestBikeDestinationValue:
        shortestBikeDestinationName = bName
        shortestBikeDestinationValue = distanceFromRandomToDestination


print("Nearest bike station to Random Origin:", shortestBikeOriginName, uniqueBikeLocations[shortestBikeOriginName])
print("Nearest bike station to Random Destination:", shortestBikeDestinationName, uniqueBikeLocations[shortestBikeDestinationName])


Nearest bike station to Random Origin: Castlehaven Road, Camden Town (51.544129600000005, -0.14408479999999999)
Nearest bike station to Random Destination: Clifton Road, Maida Vale (51.5253435, -0.1794209)


In [ ]:
# TODO!
#calculate shortest time taken to travel between 2 the 2 stations (Djikstras)
#return this value